In [1]:
# 📦 Gerekli kütüphaneleri yükle
import pandas as pd
import numpy as np
from sklearn.preprocessing import LabelEncoder


In [2]:
# 📥 Veriyi oku
df = pd.read_csv("ecommerce_clickstream_transactions.csv")  # kendi yolunu yaz


In [3]:
# 🕒 Timestamp sütununu datetime formatına çevir
df['Timestamp'] = pd.to_datetime(df['Timestamp'])


In [4]:
# 🎯 Sadece ürünle ilgili olan EventType'ları filtrele
filtered = df[df["EventType"].isin(["product_view", "add_to_cart", "purchase"])].copy()


In [5]:
# ❌ ProductID olmayan satırları çıkar
filtered.dropna(subset=["ProductID"], inplace=True)


In [6]:
# 🔢 UserID ve ProductID'yi sayısal hale getir (makine öğrenmesi için gerekli)
user_encoder = LabelEncoder()
product_encoder = LabelEncoder()

filtered["UserID_enc"] = user_encoder.fit_transform(filtered["UserID"])
filtered["ProductID_enc"] = product_encoder.fit_transform(filtered["ProductID"])


In [8]:
# 🧱 Kullanıcı-Ürün çiftine göre davranışları say
pivot_df = filtered.pivot_table(
    index=["UserID_enc", "ProductID_enc"],
    columns="EventType",        # product_view, add_to_cart, purchase
    aggfunc="size",             # her olaydan kaç kez yapmış?
    fill_value=0                # olmayanlara 0 yaz
).reset_index()


In [9]:
# 🎯 Hedef sütunu: satın almış mı? → purchase sayısı 1 veya daha fazlaysa 1, yoksa 0
pivot_df["label"] = (pivot_df["purchase"] > 0).astype(int)


In [10]:
# 👀 İlk 5 satırı gör
pivot_df.head()


EventType,UserID_enc,ProductID_enc,add_to_cart,product_view,purchase,label
0,0,100,1,0,0,0
1,0,188,1,0,0,0
2,0,386,0,1,0,0
3,0,1331,0,0,1,1
4,0,1403,1,0,0,0


In [11]:
# 📦 Makine öğrenmesi modelleri ve metrikler için kütüphaneleri yükle
from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import train_test_split
from sklearn.metrics import classification_report, confusion_matrix, accuracy_score


In [12]:
# 🎯 Özellikler (bağımsız değişkenler) ve hedef (bağımlı değişken) ayrılıyor
# Satın almayı tahmin etmek için 'product_view' ve 'add_to_cart' özelliklerini kullanıyoruz
X = pivot_df[["product_view", "add_to_cart"]]  # özellikler
y = pivot_df["label"]                          # hedef: satın almış mı? (0 veya 1)


In [13]:
# 🔀 Eğitim ve test verisi olarak ayırıyoruz (%80 eğitim, %20 test)
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)


In [14]:
# ⚙️ MODEL 1: Lojistik Regresyon modeli tanımlanıyor ve eğitiliyor
logreg = LogisticRegression()
logreg.fit(X_train, y_train)

# 🔮 Test verisi üzerinde tahmin yapılır
y_pred_logreg = logreg.predict(X_test)


In [15]:
# ⚙️ MODEL 2: Random Forest sınıflandırıcısı tanımlanıyor ve eğitiliyor
rf = RandomForestClassifier(n_estimators=100, random_state=42)
rf.fit(X_train, y_train)

# 🔮 Test verisi üzerinde tahmin yapılır
y_pred_rf = rf.predict(X_test)


In [16]:
# 📈 Modellerin doğruluk, sınıflandırma raporu ve karışıklık matrisi (confusion matrix) hesaplanır
results = {
    "Lojistik Regresyon": {
        "Accuracy": accuracy_score(y_test, y_pred_logreg),
        "Classification Report": classification_report(y_test, y_pred_logreg, output_dict=True),
        "Confusion Matrix": confusion_matrix(y_test, y_pred_logreg)
    },
    "Random Forest": {
        "Accuracy": accuracy_score(y_test, y_pred_rf),
        "Classification Report": classification_report(y_test, y_pred_rf, output_dict=True),
        "Confusion Matrix": confusion_matrix(y_test, y_pred_rf)
    }
}


In [18]:
# 🎯 Doğruluk skorlarını tablo halinde özetleyelim
import pandas as pd
summary_df = pd.DataFrame({
    "Model": ["Lojistik Regresyon", "Random Forest"],
    "Accuracy": [results["Lojistik Regresyon"]["Accuracy"], results["Random Forest"]["Accuracy"]]
})

# ✅ Doğrudan notebook çıktısı için:
summary_df


,Model,Accuracy
0,Lojistik Regresyon,0.99922
1,Random Forest,0.99922


In [19]:
# 🎯 Lojistik Regresyon için confusion matrix ve classification report
print("🔍 Lojistik Regresyon Sonuçları")
print("-" * 40)

# Confusion Matrix
print("Confusion Matrix:")
print(confusion_matrix(y_test, y_pred_logreg))

# Classification Report (precision, recall, f1-score, support)
print("\nClassification Report:")
print(classification_report(y_test, y_pred_logreg))


🔍 Lojistik Regresyon Sonuçları
----------------------------------------
Confusion Matrix:
[[4262    0]
 [   5 2142]]

Classification Report:
              precision    recall  f1-score   support

           0       1.00      1.00      1.00      4262
           1       1.00      1.00      1.00      2147

    accuracy                           1.00      6409
   macro avg       1.00      1.00      1.00      6409
weighted avg       1.00      1.00      1.00      6409



In [20]:
# 🎯 Random Forest için confusion matrix ve classification report
print("\n\n🔍 Random Forest Sonuçları")
print("-" * 40)

# Confusion Matrix
print("Confusion Matrix:")
print(confusion_matrix(y_test, y_pred_rf))

# Classification Report
print("\nClassification Report:")
print(classification_report(y_test, y_pred_rf))




🔍 Random Forest Sonuçları
----------------------------------------
Confusion Matrix:
[[4262    0]
 [   5 2142]]

Classification Report:
              precision    recall  f1-score   support

           0       1.00      1.00      1.00      4262
           1       1.00      1.00      1.00      2147

    accuracy                           1.00      6409
   macro avg       1.00      1.00      1.00      6409
weighted avg       1.00      1.00      1.00      6409

